<a href="https://colab.research.google.com/github/jhgil92/da_study/blob/master/logistic%20regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 빅쿼리 auth

project_id = 'dastudy-214803'

from google.colab import auth
import pandas as pd
auth.authenticate_user()

In [0]:
query ='''
SELECT
  *
FROM
  [logistic.docum]
'''

df = pd.io.gbq.read_gbq(query,project_id)

In [0]:
# 불필요한 칼럼 제거

del df["int64_field_0"]

In [0]:
df.head()

# label : Liberal or Science
# https://docs.google.com/spreadsheets/d/1yk1nfnOfW_W94JdL9eof5DHEadiu8HMsZfwcmlYNp4o/edit?usp=sharing
# V1 : 우주, 빛, 에너지, 물체, 원자, 전자, 질량 등, etc
# V2 : 사람, 우리, 아이, 뇌, 행동, 생각, 기억, etc
# V3 : 원소, 사용, 반응, 화합물, 금속, 동위원소, 반감기, etc
# V4 : 공자, 중국, 사람, 강호, 무공, 게임, 하늘, 나라, 판타지, etc
# V5 : 수학, 수, 식, 값, 증명, 문제, 소수, 다음, 개수, 계산, 함수, 숫자, etc
# V6 : 일러스트, 셜록, 홈즈, 작품, 콘텐츠, 저작권, 법적책임, 소설, etc
# V7 : 지구, 별, 태양, 사진, 행성, 은하, 별자리, 위성, 달, 지역, 구름, 바람, 관측, etc
# V8 : 철학, 인간, 우리, 주장, 과학, 존재, 지식, 욕망, 세계, 개념, 의미, 철학자, etc
# V9 : 책, 소설, 작품, 작가, 사랑, 시, 삶, 문학, 이야기, 서재, 시집, 인생, 시인, etc
# V10 : 몸, 유전자, 세포, 생물, 동물, 기생충, 식물, 바다, 화석, 단백질, 먹이, 바이러스, etc

,label,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,Liberal,12,8,22,4,10,5,6,11,5,4
1,Liberal,9,8,24,4,11,3,6,10,6,4
2,Liberal,8,8,15,5,8,4,5,7,5,3
3,Liberal,12,9,27,5,13,4,7,8,6,4
4,Liberal,12,9,27,6,9,3,5,8,5,7


In [0]:
df['label'].value_counts()

Liberal    1590
Science    1471
Name: label, dtype: int64

In [0]:
df.groupby(df.label).mean()

# Liberal <-> Science가 전처리 과정에서 바뀐 듯..

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
label,,,,,,,,,,
Liberal,9.024528,13.614465,6.767925,7.218239,10.434591,4.299371,6.028931,9.899371,7.340252,5.908176
Science,4.789259,14.738273,2.603671,7.997961,7.590075,8.680489,2.848402,10.522774,13.184228,3.700884


In [0]:
# logistic regression을 위해 label을 0과 1로 변경 / Liberal(real science) = 1, Science(real liberal) = 0

df.loc[df['label']== "Liberal", 'label'] = 1
df.loc[df['label']== "Science", 'label'] = 0

In [0]:
df.head()

,label,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,1,12,8,22,4,10,5,6,11,5,4
1,1,9,8,24,4,11,3,6,10,6,4
2,1,8,8,15,5,8,4,5,7,5,3
3,1,12,9,27,5,13,4,7,8,6,4
4,1,12,9,27,6,9,3,5,8,5,7


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

# train 데이터와 test 데이터 분할 (8:2)

train, test = train_test_split(df, test_size=0.2)

In [0]:
print('train data set dimension=', train.shape)
print('test data set dimension=', test.shape)

train data set dimension= (2448, 11)
test data set dimension= (613, 11)


In [0]:
train['label'].value_counts()

1    1269
0    1179
Name: label, dtype: int64

In [0]:
test['label'].value_counts()

1    321
0    292
Name: label, dtype: int64

In [0]:
# 학습 및 reshape을 위해 dataframe을 numpy로 변경

train.as_matrix()

y_data = train['label']

del train['label']

x_data = train


y_test = test['label']

del test['label']

x_test = test

In [0]:
print(x_data.shape, y_data.shape)

# y_data의 랭크 오류, 추후 학습시 오류가 발생 -> reshape을 통해 해결

(2448, 10) (2448,)


In [0]:
y_data = np.reshape(y_data, (-1,1))

y_test = np.reshape(y_test, (-1,1))

# y_data reshape

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [0]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 10])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [0]:
W = tf.Variable(tf.random_normal([10, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [0]:
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

In [0]:
cost = -tf.reduce_mean(Y*(tf.log(hypothesis+1e-4))+(1-Y)*(tf.log(1-hypothesis+1e-4)))

# https://stackoverflow.com/questions/50561959/tensorflow-cost-value-return-nan
# hypothesis가 0 or 1이 될 경우, 학습이 안되는 케이스 발생, hypothesis에 아주 작은 수를 더해줘서 문제를 해결

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

In [0]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [0]:
# 학습 및 train data 검증

with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(5001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, cost_val)

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)
    
# train data에서 정확도는 90.8%

0 3.0404558
200 0.6686325
400 0.52862686
600 0.43273613
800 0.36545494
1000 0.31873092
1200 0.28745374
1400 0.266296
1600 0.2519146
1800 0.24313848
2000 0.23770456
2200 0.23395441
2400 0.23115529
2600 0.22899964
2800 0.227332
3000 0.22605114
3200 0.22507814
3400 0.22434713
3600 0.2238023
3800 0.2233979
4000 0.22309731
4200 0.2228721
4400 0.22270092
4600 0.2225681
4800 0.22246233
5000 0.22237585

Hypothesis:  [[0.9992483 ]
 [0.5741051 ]
 [0.99217814]
 ...
 [0.9861645 ]
 [0.99999595]
 [0.9995515 ]] 
Correct (Y):  [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]] 
Accuracy:  0.9019608


In [0]:
# test data에 적용

with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(5001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, cost_val)

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_test, Y: y_test})
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)
    
# test data에서 정확도 89.5%

0 3.0404558
200 0.6686325
400 0.52862686
600 0.43273613
800 0.36545494
1000 0.31873092
1200 0.28745374
1400 0.266296
1600 0.2519146
1800 0.24313848
2000 0.23770456
2200 0.23395441
2400 0.23115529
2600 0.22899964
2800 0.227332
3000 0.22605114
3200 0.22507814
3400 0.22434713
3600 0.2238023
3800 0.2233979
4000 0.22309731
4200 0.2228721
4400 0.22270092
4600 0.2225681
4800 0.22246233
5000 0.22237585

Hypothesis:  [[4.39346321e-02]
 [1.41819182e-03]
 [9.99095321e-01]
 [6.99271506e-04]
 [9.99985337e-01]
 [9.84505832e-01]
 [1.04324082e-02]
 [1.49556703e-03]
 [3.97857010e-01]
 [1.71174500e-02]
 [4.81566004e-02]
 [8.18059564e-01]
 [9.99793112e-01]
 [1.53596438e-02]
 [9.99848247e-01]
 [3.00284624e-01]
 [3.00284624e-01]
 [9.99539256e-01]
 [1.83736414e-01]
 [3.00284624e-01]
 [1.25109896e-01]
 [6.69682920e-02]
 [2.01075245e-02]
 [6.60366192e-03]
 [8.26970220e-01]
 [3.49748251e-03]
 [1.24629610e-03]
 [2.54970968e-01]
 [9.91774678e-01]
 [4.38675076e-01]
 [9.99972224e-01]
 [9.88126755e-01]
 [4.29918796